****************************************************************

# Data exploration

In [1]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


### Quick-note on project directory

The main root dir `~/3dcorrection` is structured as follow:
* `data/` contains raw and preprocessed data. 
    * `raw/` is actually a symbolic link to the same repo for all candidates, DO NOT TOUCH IT!
    * `processed/` will be created when data is preprocessed and will contain all transformed data
* 

In [2]:
import os.path as osp
import sys

sys.path.append(osp.abspath('..'))

### The 3D Correction Use-Case

The European Centre for Medium-range Weather Forecasts (ECMWF) has developed a series of model giving the current best accurate parametrization scheme available—among those, SPARTACUS delivers **radiation** prediction over the globe. Because it is demanding in computations, a simpler, degraded model called TRIPLECLOUD is developed to satisfy the production environment constraints. 

Like most climate models, to leverage hardware acceleration, the choice is made to split the globe in blocks—this has the immediate consequence of losing the spatial correlation for a gain in parallelization. 

The unit block is a column that express values throughout the vertical dimension over a set of levels. Each level is

Now let's load the raw data we'll be using throughout this hands-on. Take a look at the [source notebook](https://git.ecmwf.int/projects/MLFET/repos/maelstrom-radiation/browse/climetlab_maelstrom_radiation/radiation.py) for a more info on the variables.

## Download dataset

In [16]:
import climetlab as cml
import dask
import dask.array as da
from glob import glob
import numpy as np
import os
import os.path as osp
import xarray as xr

import config

step = 250
# step = 1000

cml.settings.set("cache-directory", config.cache_data_path)

cmlds = cml.load_dataset(
    'maelstrom-radiation', 
    dataset='3dcorrection', 
    raw_inputs=False, 
    timestep=list(range(0, 3501, step)), 
    minimal_outputs=False,
    patch=list(range(0, 16, 1)),
    hr_units='K d-1',
)

xr_array = cmlds.to_xarray()
xr_array

<xarray.Dataset>
Dimensions:       (column: 4070400, sca_variable: 17, level: 137, col_variable: 27, half_level: 138, hl_variable: 2, p_variable: 1, level_interface: 136, inter_variable: 1)
Dimensions without coordinates: column, sca_variable, level, col_variable, half_level, hl_variable, p_variable, level_interface, inter_variable
Data variables: (12/13)
    sca_inputs    (column, sca_variable) float32 dask.array<chunksize=(16960, 1), meta=np.ndarray>
    col_inputs    (column, level, col_variable) float32 dask.array<chunksize=(16960, 137, 1), meta=np.ndarray>
    hl_inputs     (column, half_level, hl_variable) float32 dask.array<chunksize=(16960, 138, 1), meta=np.ndarray>
    pressure_hl   (column, half_level, p_variable) float32 dask.array<chunksize=(16960, 138, 1), meta=np.ndarray>
    inter_inputs  (column, level_interface, inter_variable) float32 dask.array<chunksize=(16960, 136, 1), meta=np.ndarray>
    lat           (column) float32 dask.array<chunksize=(16960,), meta=np.ndarray>
    ...            ...
    flux_dn_sw    (column, half_level) float32 dask.array<chunksize=(16960, 138), meta=np.ndarray>
    flux_up_sw    (column, half_level) float32 dask.array<chunksize=(16960, 138), meta=np.ndarray>
    flux_dn_lw    (column, half_level) float32 dask.array<chunksize=(16960, 138), meta=np.ndarray>
    flux_up_lw    (column, half_level) float32 dask.array<chunksize=(16960, 138), meta=np.ndarray>
    hr_sw         (column, level) float32 dask.array<chunksize=(16960, 137), meta=np.ndarray>
    hr_lw         (column, level) float32 dask.array<chunksize=(16960, 137), meta=np.ndarray>

In [17]:
print(f"num of instants: {3500 // step} /3500")
print(f"size: {xr_array.nbytes / float(1 << 30):,.0f} GB")

num of instants: 14 /3500
size: 77 GB


In [ ]:
Explain the idea of chunks

The returned object is a ClimateLab dataset Xarray Dataset

Let's check the content of the downloaded file

most operations are computed lazily in dask/xarray when needed and if possible on every chunk, treated and seen 'as if' it was a continuous array

In [18]:
xr_array.sca_inputs

<xarray.DataArray 'sca_inputs' (column: 4070400, sca_variable: 17)>
dask.array<concatenate, shape=(4070400, 17), dtype=float32, chunksize=(16960, 6), chunktype=numpy.ndarray>
Dimensions without coordinates: column, sca_variable
Attributes:
    long_name:  Skin_temperature
    units:      K

In [19]:
xr_array.col_inputs

,Array,Chunk
Bytes,56.09 GiB,106.36 MiB
Shape,"(4070400, 137, 27)","(16960, 137, 12)"
Count,22800 Tasks,3840 Chunks
Type,float32,numpy.ndarray


In [20]:
features = [
    'sca_inputs',
    'col_inputs',
    'hl_inputs',
    'inter_inputs',
    'flux_dn_sw',
    'flux_up_sw',
    'flux_dn_lw',
    'flux_up_lw',
]

for feat in features:
    print(f'{feat}: {xr_array[feat].data}')

sca_inputs: dask.array<concatenate, shape=(4070400, 17), dtype=float32, chunksize=(16960, 6), chunktype=numpy.ndarray>
col_inputs: dask.array<concatenate, shape=(4070400, 137, 27), dtype=float32, chunksize=(16960, 137, 12), chunktype=numpy.ndarray>
hl_inputs: dask.array<concatenate, shape=(4070400, 138, 2), dtype=float32, chunksize=(16960, 138, 1), chunktype=numpy.ndarray>
inter_inputs: dask.array<transpose, shape=(4070400, 136, 1), dtype=float32, chunksize=(16960, 136, 1), chunktype=numpy.ndarray>
flux_dn_sw: dask.array<concatenate, shape=(4070400, 138), dtype=float32, chunksize=(16960, 138), chunktype=numpy.ndarray>
flux_up_sw: dask.array<concatenate, shape=(4070400, 138), dtype=float32, chunksize=(16960, 138), chunktype=numpy.ndarray>
flux_dn_lw: dask.array<concatenate, shape=(4070400, 138), dtype=float32, chunksize=(16960, 138), chunktype=numpy.ndarray>
flux_up_lw: dask.array<concatenate, shape=(4070400, 138), dtype=float32, chunksize=(16960, 138), chunktype=numpy.ndarray>


## Flattened data

In [21]:
import utils 

dataset_len = xr_array.dims['column']
print(f"dataset len: {dataset_len}")
print(f"prime factor decomposition: {utils.prime_factors(dataset_len)}")

dataset len: 4070400
prime factor decomposition: [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 5, 5, 53]


In [ ]:
num_shards = 53 * 2 ** 4
shard_size = dataset_len // num_shards

Configure Dask to execute in a multithreated environment, as opposed to multiprocessed. Just making the implicit explicit here.

In [ ]:
dask.config.set(scheduler='threads')

In [23]:
data = {}
for feat in features:
    array = xr_array[feat].data
    array = da.reshape(array, shape=(array.shape[0], -1))
    data.update({feat: array})

In [24]:
x = da.concatenate([
    data['hl_inputs'],
    data['inter_inputs'],
    data['sca_inputs'],
    data['col_inputs']
], axis=-1)

y = da.concatenate([
    data['flux_dn_sw'],
    data['flux_up_sw'],
    data['flux_dn_lw'],
    data['flux_up_lw'],
], axis=-1)

In [25]:
x

dask.array<concatenate, shape=(4070400, 4128), dtype=float32, chunksize=(16960, 459), chunktype=numpy.ndarray>

In [26]:
x_ = da.rechunk(x, chunks=(shard_size, *x.shape[1:]))
y_ = da.rechunk(y, chunks=(shard_size, *y.shape[1:]))

In [27]:
x_

dask.array<rechunk-merge, shape=(4070400, 4128), dtype=float32, chunksize=(4800, 4128), chunktype=numpy.ndarray>

**Looks better!**

In [28]:
out_dir = osp.join(config.processed_data_path, f'flattened-{step}')

x_path, y_path = utils.purgedirs([
    osp.join(out_dir, 'x'), 
    osp.join(out_dir, 'y')
])
    
da.to_npy_stack(x_path, x_, axis=0)
da.to_npy_stack(y_path, y_, axis=0)

In [29]:
utils.save_params(
    step, 
    {
        "dtype": x.dtype.name,
        "timestep": step,
        "dataset_len": len(x),
        "num_shards": len(x.chunks[0]),
        "x_shape": x.chunksize,
        "y_shape": y.chunksize,
    }, 
    'flattened')

## Feature engineering

In [30]:
dataset_size = xr_array.dims['column']
num_shards = 53 * 2 ** 4
shard_size = dataset_size // num_shards

data = {}
for feat in features:
    array = xr_array[feat].data
    array = da.rechunk(array, chunks=(shard_size, *array.shape[1:]))
    data.update({feat: array})

In [31]:
def broadcast_features(array: da.Array):
    a = da.repeat(array, 138, axis=-1)
    a = da.moveaxis(a, -2, -1)
    return a

def pad_tensor(array: da.Array):
    a = da.pad(array, ((0, 0), (1, 1), (0, 0)))
    return a

In [32]:
from typing import Dict

# still lazy
x = da.concatenate([
    data['hl_inputs'],
    pad_tensor(data['inter_inputs']),
    broadcast_features(data['sca_inputs'][..., np.newaxis])
], axis=-1)

y = da.concatenate([
    data['flux_dn_sw'][..., np.newaxis],
    data['flux_up_sw'][..., np.newaxis],
    data['flux_dn_lw'][..., np.newaxis],
    data['flux_up_lw'][..., np.newaxis],
], axis=-1)

edge = data['col_inputs']

print(f"x of shape: {x.shape}")
print(f"y of shape: {y.shape}")
print(f"edge of shape: {edge.shape}")

x of shape: (4070400, 138, 20)
y of shape: (4070400, 138, 4)
edge of shape: (4070400, 137, 27)


In [33]:
x

dask.array<concatenate, shape=(4070400, 138, 20), dtype=float32, chunksize=(4800, 136, 17), chunktype=numpy.ndarray>

### To a single HDF5 file

In [34]:
x_ = da.rechunk(x, chunks=(shard_size, *x.shape[1:]))
y_ = da.rechunk(y, chunks=(shard_size, *y.shape[1:]))
edge_ = da.rechunk(edge, chunks=(shard_size, *edge.shape[1:]))

In [35]:
out_file = osp.join(config.processed_data_path, f'features-{step}.h5')
if osp.isfile(out_file): os.remove(out_file)
    
x_.to_hdf5(out_file, '/x')
y_.to_hdf5(out_file, '/y')
edge_.to_hdf5(out_file, '/edge')

### To a stack of NumPY files

In [36]:
out_dir = osp.join(config.processed_data_path, f'features-{step}')

x_path, y_path, edge_path = utils.purgedirs([
    osp.join(out_dir, 'x'), 
    osp.join(out_dir, 'y'), 
    osp.join(out_dir, 'edge')
])
    
da.to_npy_stack(x_path, x, axis=0)
da.to_npy_stack(y_path, y, axis=0)
da.to_npy_stack(edge_path, edge, axis=0)

### Saving parameters for later use

In [37]:
utils.save_params(
    step, 
    {
        "dtype": x.dtype.name,
        "timestep": step,
        "dataset_len": len(x),
        "num_shards": len(x.chunks[0]),
        "x_shape": x.chunksize,
        "y_shape": y.chunksize,
        "edge_shape": edge.chunksize,
    }, 
    'features')